In [ ]:
import pandas as pd
import sklearn as sk 
#343432

df = pd.read_csv('dsc_salary_2025_train.csv')

df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       16000 non-null  int64  
 1   id                               16000 non-null  int64  
 2   first_name                       16000 non-null  object 
 3   hackathons_count                 16000 non-null  int64  
 4   courses_count                    14485 non-null  float64
 5   pet_projects_count               16000 non-null  int64  
 6   leetcode_solved                  16000 non-null  int64  
 7   hours_study_per_week             16000 non-null  float64
 8   months_with_mentor               16000 non-null  int64  
 9   gpa                              14989 non-null  float64
 10  internship                       16000 non-null  object 
 11  is_mentor                        16000 non-null  object 
 12  english_level     

In [5]:
df.head()

,Unnamed: 0,id,first_name,hackathons_count,courses_count,pet_projects_count,leetcode_solved,hours_study_per_week,months_with_mentor,gpa,...,takes_part_in_informal_meetings,ml_theory_score,coding_skill,math_score,noise_feature_1,noise_feature_2,noise_feature_3,noise_feature_4,noise_feature_5,salary_class
0,0,0,Григорий,1,2.0,0,80,33.765801,1,2.500000,...,Yes,1.229054,1.000000,1.000000,-0.953963,5.012722,5,0.271076,0.512182,0
1,1,1,Адам,4,11.0,1,39,24.567281,4,3.844050,...,No,2.717371,4.790092,4.494198,-0.170397,6.152574,3,0.129255,1.217483,1
2,2,2,Максим,1,11.0,3,30,34.995525,7,3.608027,...,Yes,2.988535,2.781065,3.261280,1.553462,5.956906,6,0.281468,0.570660,0
3,3,3,Семён,4,9.0,2,26,33.092089,4,3.800498,...,Yes,3.974946,4.472813,4.536403,0.493162,4.253676,6,0.295052,-0.119912,3
4,4,4,Серафим,0,7.0,0,14,18.597681,3,NaN,...,No,5.000000,3.999612,2.804313,-0.240329,7.004080,9,0.060344,4.650848,0


In [6]:
df.tail()

,Unnamed: 0,id,first_name,hackathons_count,courses_count,pet_projects_count,leetcode_solved,hours_study_per_week,months_with_mentor,gpa,...,takes_part_in_informal_meetings,ml_theory_score,coding_skill,math_score,noise_feature_1,noise_feature_2,noise_feature_3,noise_feature_4,noise_feature_5,salary_class
15995,15995,15995,Егор,3,13.0,1,39,19.759781,8,3.349925,...,No,2.312058,2.765716,2.647615,-0.335822,6.559688,0,0.753106,0.331039,1
15996,15996,15996,Тимофей,1,4.0,2,84,13.055841,7,3.958961,...,No,4.245629,2.915494,3.271157,-2.092204,5.850191,5,0.613521,-1.278644,1
15997,15997,15997,Тихон,0,NaN,0,16,21.649260,4,3.252337,...,No,2.664337,3.751479,4.274189,-1.327510,5.407308,3,0.210406,0.696833,0
15998,15998,15998,Светлана,2,10.0,1,0,21.753878,10,3.515526,...,No,2.978224,2.950392,1.543998,0.232642,3.866242,6,0.348857,-0.749985,1
15999,15999,15999,Семён,1,8.0,2,19,42.828175,0,3.509653,...,No,2.052362,1.699804,2.009420,-1.119074,4.341436,0,0.278306,-3.921241,0
